### Yahoo Image Seach Crawler with full-scroll

<span style="color: red">**CAUTION! DO NOT execute this program on BACKGROUND**</span>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import os
import json
from urllib.request import *
import sys
import time

As you are using Selenium 3.8.0 you have to use GeckoDriver mandatorily. But again as you are using Firefox v46.0 you have to set the capability marionette to False through `DesiredCapabilities()` as follows :

```python
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True
```

**Current version**

- `geckodriver` : v0.24.0

- `FireFox` : 68.0.1

In [2]:
# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()
download_path = "images/"

In [3]:
searchtext = "gucci t shirt"
searchtext = searchtext.split(' ')
searchtext = '+'.join(searchtext)

num_requested = 500
number_of_scrolls = num_requested / 135 + 1 

In [4]:
if not os.path.exists(download_path + searchtext.replace("+", "_")):
    os.makedirs(download_path + searchtext.replace("+", "_"))

In [5]:
extensions = ["jpg", "png"] #, "gif"
ext_query = ''
for i, ex in enumerate(extensions):
    ext_query += '&FileFormat={}'.format(ex)

url = "https://sg.images.search.yahoo.com/search/images?p="+searchtext+ext_query
print(url)

# driver = webdriver.Firefox()
# driver = webdriver.Firefox(capabilities=cap) # executable_path="C:\\path\\to\\geckodriver.exe"
driver = webdriver.Firefox(executable_path="./geckodriver/geckodriver")
driver.get(url)

https://sg.images.search.yahoo.com/search/images?p=gucci+t+shirt&FileFormat=jpg&FileFormat=png


**Please wait until the browser window shows up**

In [6]:
headers = {}
headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"

img_count = 0
downloaded_img_count = 0

In [8]:
color = {0: 'all', 1:'Red', 2:'Orange', 3:'Yellow', 4:'Green', 5:'Teal', 6:'Blue', 
         7:'Purple', 8:'Pink', 9:'Brown', 10:'Black', 11:'Grey', 12:'White', 13: 'Black & White'}

choice = 12 # choose wanted color!

if choice != 0:
    try:
        driver.find_element_by_xpath('//div[@id="filters"]/section/ul/li/ul/li/a[@tt="{}"]'.format(color[choice])).click()
    except Exception as e:
        print (f"Color Selection Failed: ")
        print(e)
        pass
    
time.sleep(1)
# driver.quit()

**Please wait until reloading is completed**

In [9]:
for _ in range(int(number_of_scrolls)):
    for __ in range(10):
        driver.execute_script("window.scrollBy(0, 1000000)")
        time.sleep(0.2)
    # to load next images
    time.sleep(0.8)
    try:
        driver.find_element_by_xpath('//*[@id="results"]/button').click()
    except Exception as e:
        print (f"Less images found: {e}")
        print ("if it seems that scrolling is done, just pass to next step.")
        break

**Please wait until scrolling is done**

In [10]:
num_imgs = driver.find_elements_by_xpath('//div[contains(@id,"res-cont")]/section/div/ul/li')
num_imgs = len(num_imgs)
print ("Total images: {}\n".format(num_imgs))

Total images: 639



In [11]:
for i in range(num_imgs):
    
    print(f"Loading resitem-{i}")
    try:
        driver.find_element_by_xpath('//li[@id="resitem-{}"]'.format(i)).click()
        if i == 0:
            time.sleep(1.5)
        else:
            time.sleep(0.3)
        
        img = driver.find_element_by_xpath('//div[@id="res-cont"]/section[@id="results"] \
                                             /div[@class="sres-cntr"]/ul[@id="sres"]/li[@class="viewer focused"] \
                                             /div[@class="cont"]/div[@class="img"]/div[@class="holder"]/div[@class="iholder"]/img')
        img_url = img.get_attribute('src')
        img_type = img_url.split('.')[-1]
        time.sleep(0.1)
        driver.find_element_by_xpath('//button[@title="Close"]').click()
        time.sleep(0.3)
        # if connection is not secured (SSL), skip the image
        if not img_url.startswith('https://'):
            print("connection is not secure")
            continue 
        img_count += 1
    except Exception as e:
        print(f"Loading failed {i} : {e}")
        continue
        
    print (f"Downloading {img_count}th image:{img_url}")
    
    try:
        if img_type not in extensions:
            img_type = "jpg"
        req = Request(img_url, headers=headers)
        raw_img = urlopen(req).read()
        f = open(download_path+searchtext.replace("+", "_")+"/"+str(downloaded_img_count)+"."+img_type, "wb")
        f.write(raw_img)
        f.close
        downloaded_img_count += 1
        time.sleep(0.3)
    except Exception as e:
        print (f"Download failed {i} : {e}")
        continue
        
    if downloaded_img_count >= num_requested:
        break

print("download complete.")
print("Total downloaded: {}/{}".format(downloaded_img_count,img_count))
driver.quit()

Loading resitem-0
Loading resitem-1
Loading resitem-2
Loading resitem-3
connection is not secure
Loading resitem-4
Loading resitem-5
Loading resitem-6
connection is not secure
Loading resitem-7
Loading resitem-8
Loading resitem-9
Loading resitem-10
Loading resitem-11
Loading resitem-12
connection is not secure
Loading resitem-13
Loading resitem-14
connection is not secure
Loading resitem-15
Loading resitem-16
Loading resitem-17
Loading resitem-18
Loading resitem-19
Loading resitem-20
connection is not secure
Loading resitem-21
Loading resitem-22
Loading resitem-23
Loading resitem-24
Loading resitem-25
Loading resitem-26
Loading resitem-27
Loading resitem-28
connection is not secure
Loading resitem-29
Loading resitem-30
connection is not secure
Loading resitem-31
Loading resitem-32
Loading resitem-33
Loading resitem-34
connection is not secure
Loading resitem-35
connection is not secure
Loading resitem-36
Loading resitem-37
Loading resitem-38
Loading resitem-39
Loading resitem-40
connec

---
---

#### **queries**

```python
image = driver.find_element_by_xpath('//div[@id="res-cont"]/section/div/ul/li[@class="viewer focused"]/div/div/div/div[@class="iholder"]/img')
image = driver.find_element_by_xpath('//div[contains(@class,"iholder")]/img')
imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
image_url = driver.find_element_by_xpath(('//div[@id="res-cont"]/section/div/ul/li[@class="viewer focused"]')
```